# Titanic Example

Here's an example training a neural net on Kaggle's Titanic dataset using the fastai and pytorch frameworks adapted from this [tutorial](https://www.kaggle.com/code/hitesh1724/titanic-1-fastai-beginner-tutorial).

In [1]:
import pandas as pd
import os

import fastcore
import fastai

from fastai.tabular.all import *

### Prep The Data

First import the train and test data obtained from Kaggle

In [2]:
df_test = pd.read_csv('./data/test.csv')
df_train = pd.read_csv('./data/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We can take a statistical look into the data.

In [3]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Calculating the average null values we have in our data. It is important to know. Just to get intuition about data

In [4]:
df_train.isnull().sum().sort_index()/len(df_train)

Age            0.198653
Cabin          0.771044
Embarked       0.002245
Fare           0.000000
Name           0.000000
Parch          0.000000
PassengerId    0.000000
Pclass         0.000000
Sex            0.000000
SibSp          0.000000
Survived       0.000000
Ticket         0.000000
dtype: float64

Let's look at the datatypes of our columns to better unserstand which are continous (floats), discrete (integers), or categorial (objects)

In [5]:
df_train.dtypes
g_train =df_train.columns.to_series().groupby(df_train.dtypes).groups
g_train

{int64: ['PassengerId', 'Survived', 'Pclass', 'SibSp', 'Parch'], float64: ['Age', 'Fare'], object: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']}

Based on this we should group the names:

In [6]:
cat_names  = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
cont_names = ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare']

### Preprocessing

Now we are diving into fastai.

Here we are using fastai TabularPandas library. Which will do all the preprocessing for us. before that splitting our data into validation set to have a fair amount of idea that we are not overfitting the data. valid_pct= 0.2 means (as you may have guessed by now) it means 20% validation data.


In [7]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df_train))

to = TabularPandas(df_train, procs=[Categorify, FillMissing, Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names = 'Survived',
                   splits=splits)

Here you can see 'Age_na', 'WikiId_na', 'Age_wiki_na', 'Class_na' . Which were created becoz their columns had missing values

In [9]:
g_train =to.train.xs.columns.to_series().groupby(to.train.xs.dtypes).groups
g_train

{int8: ['Sex', 'Embarked', 'Age_na'], int16: ['Name', 'Ticket', 'Cabin'], float64: ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare']}

In [10]:
to.train.xs.Age_na.head()

102    1
859    2
802    1
516    1
296    1
Name: Age_na, dtype: int8

In [11]:
to.train.xs

,Name,Sex,Ticket,Cabin,Embarked,Age_na,PassengerId,Pclass,SibSp,Parch,Age,Fare
102,859,2,418,99,3,1,-1.323826,-1.561356,-0.475085,0.783027,-0.657612,0.835807
859,673,2,179,0,1,2,1.619348,0.827586,-0.475085,-0.468763,-0.047492,-0.499267
802,143,2,34,48,3,1,1.397735,-1.561356,0.435494,2.034818,-1.420263,1.649763
516,473,1,561,143,3,1,0.285783,-0.366885,-0.475085,-0.468763,0.333834,-0.436937
296,325,2,213,0,1,1,-0.569565,0.827586,-0.475085,-0.468763,-0.466950,-0.499267
...,...,...,...,...,...,...,...,...,...,...,...,...
272,531,1,163,0,3,1,-0.662876,-0.366885,-0.475085,0.783027,0.867689,-0.265427
200,835,2,307,0,3,1,-0.942808,0.827586,-0.475085,-0.468763,-0.123757,-0.455994
49,39,1,378,0,3,1,-1.529887,0.827586,0.435494,-0.468763,-0.886408,-0.297824
41,828,1,54,0,3,1,-1.560991,-0.366885,0.435494,-0.468763,-0.200022,-0.236843


### Training

Now that our data is preprocessed we can use the RandomForestClassifier to solve this problem.

In [12]:
from sklearn.ensemble import RandomForestClassifier

X_train = to.train.xs
X_valid = to.valid.xs

y_train = to.train.ys.values.ravel()
y_valid = to.valid.ys.values.ravel()

We have table without any hardcore preprocessing all we did was just to use fastai tabular function to get this.

In [13]:
X_train.head()

,Name,Sex,Ticket,Cabin,Embarked,Age_na,PassengerId,Pclass,SibSp,Parch,Age,Fare
102,859,2,418,99,3,1,-1.323826,-1.561356,-0.475085,0.783027,-0.657612,0.835807
859,673,2,179,0,1,2,1.619348,0.827586,-0.475085,-0.468763,-0.047492,-0.499267
802,143,2,34,48,3,1,1.397735,-1.561356,0.435494,2.034818,-1.420263,1.649763
516,473,1,561,143,3,1,0.285783,-0.366885,-0.475085,-0.468763,0.333834,-0.436937
296,325,2,213,0,1,1,-0.569565,0.827586,-0.475085,-0.468763,-0.466950,-0.499267


In [14]:
rnf_classifier= RandomForestClassifier(n_estimators=100, n_jobs=-1)
rnf_classifier.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1)

We just Trained randomforest classifier and predicting accuracy on validation set

In [15]:
from sklearn.metrics import accuracy_score

y_pred = rnf_classifier.predict(X_valid)
accuracy_score(y_pred, y_valid)

0.8202247191011236

### TEST Dataset

In [16]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Doing the same preprocessing as before:

In [17]:
df_test.dtypes
g_train =df_test.columns.to_series().groupby(df_test.dtypes).groups
g_train

{int64: ['PassengerId', 'Pclass', 'SibSp', 'Parch'], float64: ['Age', 'Fare'], object: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']}

In [18]:
cat_names  = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
cont_names = ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare']

In [19]:
test = TabularPandas(df_test, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   )

In [20]:
X_test= test.train.xs


In [21]:
X_test.head()

,Name,Sex,Ticket,Cabin,Embarked,Age_na,Fare_na,PassengerId,Pclass,SibSp,Parch,Age,Fare
0,207,2,153,0,2,1,1,-1.727912,0.873482,-0.499470,-0.400248,0.386231,-0.497413
1,404,1,222,0,3,1,1,-1.719625,0.873482,0.616992,-0.400248,1.371370,-0.512278
2,270,2,74,0,2,1,1,-1.711337,-0.315819,-0.499470,-0.400248,2.553537,-0.464100
3,409,2,148,0,3,1,1,-1.703050,0.873482,-0.499470,-0.400248,-0.204852,-0.482475
4,179,1,139,0,3,1,1,-1.694763,0.873482,0.616992,0.619896,-0.598908,-0.417491


In [22]:
X_test.dtypes
g_train =X_test.columns.to_series().groupby(X_test.dtypes).groups
g_train


{int8: ['Sex', 'Cabin', 'Embarked', 'Age_na', 'Fare_na'], int16: ['Name', 'Ticket'], float64: ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare']}

In [23]:
X_test= X_test.drop('Fare_na', axis=1)

In [24]:
y_pred=rnf_classifier.predict(X_test)

In [25]:
y_pred= y_pred.astype(int)

In [26]:
output= pd.DataFrame({'PassengerId':df_test.PassengerId, 'Survived': y_pred})
output.to_csv('my_submission_titanic.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
